In [1]:
test_fold=4 #choose fold from[0,1,2,3,4] 

In [2]:
# !pip install openpyxl 
!pip install xlrd==2.0.1 
!pip install Pandas==1.3.5
# !pip install Pandas==1.1.5
!pip install  gdown
!pip install deepchem
!pip install pysmiles

import tensorflow as tf

import numpy as np
import pandas as pd
 



def normalize1(X, means1=None, std1=None, means2=None, std2=None, feat_filt=None, norm='tanh_norm'):
    if std1 is None:
        std1 = np.nanstd(X, axis=0)
    if feat_filt is None:
        feat_filt = std1!=0
    X = X[:,feat_filt]
    X = np.ascontiguousarray(X)
    if means1 is None:
        means1 = np.mean(X, axis=0)
    X = (X-means1)/std1[feat_filt]
    if norm == 'norm':
        return(X, means1, std1, feat_filt)
    elif norm == 'tanh':
        return(np.tanh(X), means1, std1, feat_filt)
    elif norm == 'tanh_norm':
        X = np.tanh(X)
        if means2 is None:
            means2 = np.mean(X, axis=0)
        if std2 is None:
            std2 = np.std(X, axis=0)
        X = (X-means2)/std2
        X[:,std2==0]=0
        return(X, means1, std1, means2, std2, feat_filt) 


    
def get_data():
  !gdown https://drive.google.com/uc?id=1m69fQ5l7q3dwQeud5M1nUVNhmrTGFbZc
  smiles=pd.read_excel('pubchem.xls', header=None)
  smiles=np.array(smiles)


  !gdown 10ztxKtGSVU7p9yPoCsTnapVLThWpjjYT
  data_to_repeat=pd.read_excel('oneil.xlsx', header=None)
  data_to_repeat=np.array(data_to_repeat)
  

  !gdown https://drive.google.com/uc?id=1m69fQ5l7q3dwQeud5M1nUVNhmrTGFbZc
  unique_drugs=pd.read_excel('pubchem.xls', header=None)
  unique_drugs=np.array(unique_drugs)         
    
  !gdown 15yf5KKQqRROygqB05MZqfztmf5elTBSw
  feature_cell=pd.read_csv('vae_expression256.csv')  
  feature_cell=np.array(feature_cell)
    
  !gdown  14upx46iIPcO80y_tcvbaS3uwMkZfKLVw
  redkit_drug=pd.read_excel('redkit_drug.xlsx',header=None)
  unique_drugs1=np.array(redkit_drug)
    
  !gdown 1-qSA_3RF6eBSgSckcV5u1dORTBTia-lq
  graph=pd.read_excel('views0b123.xlsx',header=None)
  graph=np.array(graph)
  
  !gdown 1A_E0ek950T1r8OAH2EqLTdNKzIWv7K6_
  target=pd.read_csv('drug_target.csv')
  target=np.array(target)

  !gdown 1zDRCKuJUkAxATHjzWFEUc20o_4u4nDwJ
  copy=pd.read_csv('vae_copy256.csv')
  copy=np.array(copy)

  !gdown 1dQm1MQp-qh-jTweXq81HJ7zgjfYxqKyg
  mutation=pd.read_csv('vae_mutation256.csv')
  mutation=np.array(mutation)
    
  !gdown 1BV5XFD8KZB7sYLYx4YQ0uww_FhFstkru
  protematics=pd.read_csv('vae_promatics256.csv')
  protematics=np.array(protematics)

  !gdown  1zPeJOMpLEcmgKhUANm8clSIfghvSCyrC
  finger=pd.read_csv('drug_finger.csv')
  finger=np.array(finger)


  return smiles[1:,1],data_to_repeat[1:,:],unique_drugs[1:,0],feature_cell,redkit_drug,graph,unique_drugs1,target,copy,mutation,protematics,finger


def repeat_smiles1(data_to_repeat,unique_drugs,feature,unique_cell,finger_drug,graph,target,copy,mutation,protematics):
#   unique_finger_feature=finger_drug[:,1:]
#   unique_finger_name=finger_drug[:,0]
  unique_feature=unique_cell[:,1:]
  unique_name=unique_cell[:,0]
  d1=data_to_repeat[:,0]
  d2=data_to_repeat[:,1]
  c=data_to_repeat[:,2]
  unique_drugs=feature[:,0]
  feature=feature[:,1:]  
  f_drug1=[]
  f_drug2=[]
  feature_cell=[]
  finger_d1=[]
  finger_d2=[]
  graph1=[]
  graph2=[]
  target1=[]
  target2=[]
  ccopy=[]
  cmutation=[]
  cprotematics=[]

  for i in range(len(d1)):
    n1=d1[i]
    n2=d2[i]
    cc=c[i]
    k1= [m for m, v in enumerate(unique_drugs) if n1 in v]
    k2=[m for m, v in enumerate(unique_drugs) if n2 in v]
    cc1=[m for m, v in enumerate(unique_name) if cc in v]
#     r1= [m for m, v in enumerate(unique_redkit_name) if n1 in v]
#     r2=[m for m, v in enumerate(unique_redkit_name) if n2 in v]
    f_drug1.append(feature[k1[0]])
    f_drug2.append(feature[k2[0]])
    graph1.append(graph[k1[0]])
    graph2.append(graph[k2[0]])
    finger_d1.append(finger_drug[k1[0]])
    finger_d2.append(finger_drug[k2[0]])
    target1.append(target[k1[0]])
    target2.append(target[k2[0]])
    
    feature_cell.append(unique_feature[cc1[0]])
    ccopy.append(copy[cc1[0]])
    cmutation.append(mutation[cc1[0]])
    cprotematics.append(protematics[cc1[0]])

  return f_drug1,f_drug2,feature_cell,finger_d1,finger_d2,graph1,graph2,target1,target2,ccopy,cmutation,cprotematics




def train_test_input(f_drug1,f_drug2,cell_line,index_train,index_test,synery,class1,finger_d1,finger_d2,graph1,graph2,target1,target2,copy,mutation,protematics):
  train_f_drug1=[]
  train_f_drug2=[]
  train_cell_line=[]
  train_synergy=[]
  train_class=[]
  train_finger_d1=[]
  train_finger_d2=[]
  test_f_drug1=[]
  test_f_drug2=[]
  test_cell_line=[]
  test_synergy=[]
  test_class=[]
  test_finger_d1=[]
  test_finger_d2=[]
  train_graph1=[]
  test_graph1=[]
  train_graph2=[]
  test_graph2=[]
    
  train_target1=[]
  train_target2=[]
  train_copy=[]
  train_mutation=[]
  train_protematics=[]
  test_target1=[]
  test_target2=[]
  test_copy=[]
  test_mutation=[]
  test_protematics=[]
  for i in range(len(index_train)):
      
      train_f_drug1.append(f_drug1[index_train[i]])
      train_f_drug2.append(f_drug2[index_train[i]])
      train_cell_line.append(cell_line[index_train[i]])
      train_synergy.append(synergy[index_train[i]])
      train_class.append(class1[index_train[i]])
      train_finger_d1.append(finger_d1[index_train[i]])
      train_finger_d2.append(finger_d2[index_train[i]])
      train_graph1.append(graph1[index_train[i]])
      train_graph2.append(graph2[index_train[i]])
     
      train_target1.append(target1[index_train[i]]) 
      train_target2.append(target2[index_train[i]])
      train_copy.append(copy[index_train[i]])
      train_mutation.append(mutation[index_train[i]])
      train_protematics.append(protematics[index_train[i]])
 
  for ii in range(len(index_test)):
      
      test_f_drug1.append(f_drug1[index_test[ii]])
      test_f_drug2.append(f_drug2[index_test[ii]])
      test_cell_line.append(cell_line[index_test[ii]])
      test_synergy.append(synergy[index_test[ii]])
      test_class.append(class1[index_test[ii]])
      test_finger_d1.append(finger_d1[index_test[ii]])
      test_finger_d2.append(finger_d2[index_test[ii]])
      test_graph1.append(graph1[index_test[ii]])
      test_graph2.append(graph2[index_test[ii]])
        
      test_target1.append(target1[index_test[ii]])
      test_target2.append(target2[index_test[ii]])
      test_copy.append(copy[index_test[ii]])
      test_mutation.append(mutation[index_test[ii]])
      test_protematics.append(protematics[index_test[ii]])

  return train_f_drug1,train_f_drug2,train_cell_line,test_f_drug1,test_f_drug2,test_cell_line,train_synergy,train_class,test_synergy,test_class,train_finger_d1,train_finger_d2,test_finger_d1,test_finger_d2,train_graph1,test_graph1,train_graph2,test_graph2,train_target1,train_target2,train_copy,train_mutation,train_protematics,test_target1,test_target2,test_copy,test_mutation,test_protematics



def preprocess(index_train,index_test):
    index_train1=[]
    index_test1=[]
    index_train2=(index_train)[0]
    index_test2=(index_test)[0]
    for i in range(len((index_train2))):
        index_train1.append((index_train2[i]))
        
    for ii in range(len(index_test2)):
        index_test1.append((index_test2[ii]))
        
    return index_train1,index_test1

def get_data_me2(s):
    
    

    !gdown 1C7Z2ziPdQVzH3omIdIfyJa7VUmog4IIk
    labels = pd.read_csv('oneil.csv', index_col=0) 
    
    h=len(np.array(labels))
    #labels are duplicated for the two different ways of ordering in the data
    labels = pd.concat([labels, labels]) 
    
    test_fold =s
   
    idx_train = np.where(labels['fold']!=test_fold)
    

    idx_test = np.where(labels['fold']==test_fold)
#     
#    
    return idx_train,idx_test





def convert_tobin(cc):
    cb=[]
    for i in range(len(cc)):
        if(cc[i]>=0.5):
            cb.append(1)
        else:
            cb.append(0)
    return cb


def norm1(train_cell_line,test_cell_line,norm="tanh_norm"):
# norm = "norm"
    if norm == "tanh_norm":
        train_cell_line, mean, std, mean2, std2, feat_filt = normalize1(train_cell_line, norm=norm)
        test_cell_line, mean, std, mean2, std2, feat_filt = normalize1(test_cell_line, mean, std, mean2, std2, 
                                                              feat_filt=feat_filt, norm=norm)
    else:
        train_cell_line, mean, std, feat_filt = normalize1(train_cell_line, norm=norm)
        test_cell_line, mean, std, feat_filt = normalize1(test_cell_line, mean, std, feat_filt=feat_filt, norm=norm)
    
    return train_cell_line,test_cell_line




    
     

smiles,data_to_repeat,unique_drugs,unique_cell,redkit_drug,graph,feature,target,copy,mutation,protematics,finger=get_data()



data_to_repeat=np.r_[data_to_repeat,data_to_repeat]
l=int((data_to_repeat.shape[0])/2)
data_to_repeat[l:,0]=data_to_repeat[0:l,1]
data_to_repeat[l:,1]=data_to_repeat[0:l,0]
synergy=data_to_repeat[:,3]

class1=[]
for i in range(len(synergy)):
 if(synergy[i]>=30):
    class1.append(1)
 elif(synergy[i]<0): 
    class1.append(0)
 else:
    class1.append(2)    


f_drug1,f_drug2,feature_cell,finger_d1,finger_d2,graph1,graph2,target1,target2,copy,mutation,protematics=repeat_smiles1(data_to_repeat,unique_drugs,feature,unique_cell,finger,graph,target,copy,mutation,protematics)




index_train,index_test=get_data_me2(test_fold)

index_train,index_test=preprocess(index_train,index_test)
train_f_drug1,train_f_drug2,train_cell_line,test_f_drug1,test_f_drug2,test_cell_line,train_synergy,train_class,test_synergy,test_class,train_finger_d1,train_finger_d2,test_finger_d1,test_finger_d2,train_graph1,test_graph1,train_graph2,test_graph2,train_target1,train_target2,train_copy,train_mutation,train_protematics,test_target1,test_target2,test_copy,test_mutation,test_protematics=train_test_input(f_drug1,f_drug2,feature_cell,index_train,index_test,synergy,class1,finger_d1,finger_d2,graph1,graph2,target1,target2,copy,mutation,protematics)
train_cell_line=np.array(train_cell_line).astype(float)
test_cell_line=np.array(test_cell_line).astype(float)  
train_f_drug1=np.array(train_f_drug1).astype(float)
test_f_drug1=np.array(test_f_drug1).astype(float)
train_f_drug2=np.array(train_f_drug2).astype(float)
test_f_drug2=np.array(test_f_drug2).astype(float)

train_graph1=np.array(train_graph1).astype(float)
test_graph1=np.array(test_graph1).astype(float) 
train_graph2=np.array(train_graph2).astype(float)
test_graph2=np.array(test_graph2).astype(float)

train_finger_d1=np.array(train_finger_d1).astype(float)
train_finger_d2=np.array(train_finger_d2).astype(float) 
test_finger_d1=np.array(test_finger_d1).astype(float)
test_finger_d2=np.array(test_finger_d2).astype(float)


# train_target1=np.array(train_target1).astype(float)
# train_target2=np.array(train_target2).astype(float)
train_copy=np.array(train_copy).astype(float)
train_mutation=np.array(train_mutation).astype(float)
train_protematics=np.array(train_protematics).astype(float)
# test_target1=np.array(test_target1).astype(float)
# test_target2=np.array(test_target2).astype(float)
test_copy=np.array(test_copy).astype(float)
test_mutation=np.array(test_mutation).astype(float)
test_protematics=np.array(test_protematics).astype(float)

train_finger_d1,test_finger_d1=norm1(train_finger_d1,test_finger_d1)
train_finger_d2,test_finger_d2=norm1(train_finger_d2,test_finger_d2)



train_cell_line,test_cell_line=norm1(train_cell_line,test_cell_line)

train_f_drug1,test_f_drug1=norm1(train_f_drug1,test_f_drug1)
train_f_drug2,test_f_drug2=norm1(train_f_drug2,test_f_drug2)

train_graph1,test_graph1=norm1(train_graph1,test_graph1)
train_graph2,test_graph2=norm1(train_graph2,test_graph2)

# train_target1,test_target1=norm1(train_target1,test_target1)
# train_target2,test_target1=norm1(train_target2,test_target2)

train_copy,test_copy=norm1(train_copy,test_copy)
train_mutation,test_mutation=norm1(train_mutation,test_mutation)
train_protematics,test_protematics=norm1(train_protematics,test_protematics)


  Obtaining dependency information for xlrd==2.0.1 from https://files.pythonhosted.org/packages/a6/0c/c2a72d51fe56e08a08acc85d13013558a2d793028ae7385448a6ccdfae64/xlrd-2.0.1-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 5.2 MB/s eta 0:00:00
  Obtaining dependency information for Pandas==1.3.5 from https://files.pythonhosted.org/packages/ff/7a/1ce22f0f009ee31878f717bd5b3221e993a7ebc02391d7a315982c2224dc/pandas-1.3.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 72.9 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: Pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not 

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Downloading...
From: https://drive.google.com/uc?id=1m69fQ5l7q3dwQeud5M1nUVNhmrTGFbZc
To: /kaggle/working/pubchem.xls
100%|██████████████████████████████████████| 27.1k/27.1k [00:00<00:00, 67.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=10ztxKtGSVU7p9yPoCsTnapVLThWpjjYT
To: /kaggle/working/oneil.xlsx
100%|█████████████████████████████████████████| 850k/850k [00:00<00:00, 114MB/s]
Downloading...
From: https://drive.google.com/uc?id=1m69fQ5l7q3dwQeud5M1nUVNhmrTGFbZc
To: /kaggle/working/pubchem.xls
100%|██████████████████████████████████████| 27.1k/27.1k [00:00<00:00, 63.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=15yf5KKQqRROygqB05MZqfztmf5elTBSw
To: /kaggle/working/vae_expression256.csv
100%|█████████████████████████████████████████| 197k/197k [00:00<00:00, 114MB/s]
Downloading...
From: https://drive.google.com/uc?id=14upx46iIPcO80y_tcvbaS3uwMkZfKLVw
To: /kaggle/working/redkit_drug.xlsx
100%|██████████████████████████████████████| 68.6k/68.6k [00:00<00:00, 9

In [3]:
!pip install tensorflow==2.13.0
!pip install spektral
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Minimum,Maximum,Add,Maximum,PReLU, Flatten,Reshape,Dropout, Input,Dense,Add,concatenate,BatchNormalization, Activation,Lambda#,MultiHeadAttention,AdditiveAttention
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.metrics import confusion_matrix, mean_squared_error,mean_absolute_error,r2_score#,AUC
from scipy.stats import pearsonr,spearmanr
from sklearn.metrics import roc_curve,auc,accuracy_score,precision_score,cohen_kappa_score,precision_recall_curve,average_precision_score,roc_auc_score
from tensorflow.keras import regularizers
np.random.seed(10)

# train_cell_line=tf.convert_to_tensor(train_cell_line)
# test_cell_line=tf.convert_to_tensor(test_cell_line)
train_synergy=tf.convert_to_tensor(train_synergy)
train_class=tf.convert_to_tensor(train_class)
test_synergy=tf.convert_to_tensor(test_synergy)
test_class=tf.convert_to_tensor(test_class)




  Obtaining dependency information for spektral from https://files.pythonhosted.org/packages/ae/4c/b1149deb49c48d58bfc2b68fe6fc502d6c896c69eaf9997de45ad24b6fca/spektral-1.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/b0/f4/d67c8c39efe3c45dfd32bb2a3fc49cbbe5496e575cc42b8bac60fe7b6701/numpy-1.22.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 65.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed

In [4]:


def scaled_dot_product_attention(q, k, v, mask=None):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits)#, axis=-1)#tf.nn.linear()#, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output,attention_weights 

#multi-head

class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model,activation='relu',use_bias='true')
    self.wk = tf.keras.layers.Dense(d_model,activation='relu',use_bias='true')
    self.wv = tf.keras.layers.Dense(d_model,activation='relu',use_bias='true')

    self.dense = tf.keras.layers.Dense(d_model)#,activation='relu',use_bias='true')

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask=None):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
#     q=PReLU()(q)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
#     k=PReLU()(k)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
#     v=PReLU()(v)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output#, attention_weights



class CrossStitch(tf.keras.layers.Layer):

    """Cross-Stitch implementation according to arXiv:1604.03539
    Implementation adapted from https://github.com/helloyide/Cross-stitch-Networks-for-Multi-task-Learning"""

    def __init__(self, num_tasks, *args, **kwargs):
        """initialize class variables"""
        self.num_tasks = num_tasks
        super(CrossStitch, self).__init__(**kwargs)

    def build(self, input_shape):
        """initialize the kernel and set the instance to 'built'"""
        self.kernel = self.add_weight(name="kernel",
                                      shape=(self.num_tasks,
                                             self.num_tasks),
                                      initializer='identity',
                                      trainable=True)
        super(CrossStitch, self).build(input_shape)

    def call(self, xl):
        """
        called by TensorFlow when the model gets build. 
        Returns a stacked tensor with num_tasks channels in the 0 dimension, 
        which need to be unstacked.
        """
        if (len(xl) != self.num_tasks):
            # should not happen
            raise ValueError()

        out_values = []
        for this_task in range(self.num_tasks):
            this_weight = self.kernel[this_task, this_task]
            out = tf.math.scalar_mul(this_weight, xl[this_task])
            for other_task in range(self.num_tasks):
                if this_task == other_task:
                    continue  # already weighted!
                other_weight = self.kernel[this_task, other_task]
#                 out += tf.math.scalar_mul(other_weight, xl[other_task])
            out_values.append(out)
        # HACK!
        # unless we stack, and then unstack the tensors, TF (2.0.0) can't follow
        # the graph, so it aborts during model initialization.
        # return tf.stack(out_values, axis=0)
        return out_values[0],out_values[1]

    def compute_output_shape(self, input_shape):
        return [self.num_tasks] + input_shape

    def get_config(self):
        """implemented so keras can save the model to json/yml"""
        config = {
            "num_tasks": self.num_tasks
        }
        base_config = super(CrossStitch, self).get_config()
        return dict(list(config.items()) + list(base_config.items()))

    
 







In [5]:

def generate_network_att1(drug1,drug2,cell):
    # fill the architecture params from dict

    
    cell_layers = [1024,512,128]
    drop=0.0
#     snp_layers = [512,128]

#     ddi_layers=[1024,512,256]
    t_layers=[567,128]
    g_layers=[254,128]
#     g_layers=[327,128]
    dsn1_layers = [394,128]
    dsn2_layers = [394,128]
    f_layers=[142,128]
    
    c_layers=[1024,512,256]
    
    m_layers=[209,128]
    p_layers=[1024,512,256]
    
    l2_reg = 1e-3  # L2 regularization rate

#     p_layers=[drug1.shape[1],512,128]

    dropout=0.0

   
    # contruct two parallel networks
   
    drug1=Input(shape=(drug1.shape[1],))
    
    for layer in range(len(p_layers)):
      if layer == 0:

        train = Dense(int(p_layers[layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(drug1)
        train = Dropout(float(drop))(train)
        
      elif layer == (len(p_layers)-1):
        drug11 = Dense(int(p_layers[layer]), activation='linear',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(train)

      else:
        train = Dense(int(p_layers[layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(train)
        train = Dropout(float(drop))(train)
        
        
    drug2=Input(shape=(drug2.shape[1],))
    
    for layer in range(len(p_layers)):
      if layer == 0:

        train = Dense(int(p_layers[layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(drug2)
        train = Dropout(float(drop))(train)
        
      elif layer == (len(p_layers)-1):
        drug22 = Dense(int(p_layers[layer]), activation='linear',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(train)

      else:
        train = Dense(int(p_layers[layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(train)
        train = Dropout(float(drop))(train)
        
        
    cell=Input(shape=(cell.shape[1],),name='train')
    
    for layer in range(len(c_layers)):
      if layer == 0:

        train = Dense(int(c_layers[layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(cell)
        train = Dropout(float(drop))(train)
        
      elif layer == (len(c_layers)-1):
        cell1 = Dense(int(c_layers[layer]), activation='linear',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(train)

      else:
        train = Dense(int(c_layers[layer]), activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(train)
        train = Dropout(float(drop))(train)
 

  
    concatModel=concatenate([drug11,drug22,cell1])
 
    
    layer1 =MultiHeadAttention(d_model=concatModel.shape[1], num_heads=4)
    a_task1= layer1(concatModel,concatModel,concatModel)
    layer2 = MultiHeadAttention(d_model=concatModel.shape[1], num_heads=4)
    a_task2= layer2(concatModel,concatModel,concatModel)
    task11 = Reshape([a_task1.shape[2]])(a_task1)
    task22 = Reshape([a_task2.shape[2]])(a_task2)
    task1=concatenate([task11,concatModel])
    task2=concatenate([task22,concatModel])
    
    r_task1,r_task2 = CrossStitch(2)([task1,task2])

    
    r_task1=Dense(2048,activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(r_task1)
    
    r_task2=Dense(2048, activation='relu',use_bias=True,kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(r_task2)

                             
    r_task1,r_task2 = CrossStitch(2)([r_task1,r_task2])

    
    r_task1=concatenate([r_task1,task1])
    r_task2=concatenate([r_task2,task2])


    r_task1 = Dense(1024, activation='linear',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(r_task1)
    r_task1=PReLU()(r_task1)
    r_task2 = Dense(1024, activation='relu',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001))(r_task2)
    
    r_task1 = Dense(128, activation='linear',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001),name='fsynergy1')(r_task1)
    r_task1=PReLU()(r_task1)
    r_task2 = Dense(128, activation='relu',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001),name='fclass1')(r_task2)

   
    r_task1 = Dense(64, activation='linear',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001),name='fsynergy2')(r_task1)
    r_task1=PReLU()(r_task1)
    r_task2 = Dense(64, activation='relu',kernel_initializer="he_normal",kernel_regularizer=regularizers.L1(0.001),
                     activity_regularizer=regularizers.L2(0.001),name='fclass2')(r_task2)


    
    snp_output1 = Dense(1, activation='linear',name='synergy')(r_task1)
    snp_output2 = Dense(3, activation='sigmoid',name='class')(r_task2)
    

    model = Model(inputs=[drug1,drug2,cell],outputs= [snp_output1,snp_output2])

    print(model.summary())
    return model


def trainer_att1(model, train_d1,train_d2,train_c,train_synergy,train_class, epo, batch_size, earlyStop,test_d1,test_d2,test_c,test_synergy,test_class1):

    optimizer = tf.keras.optimizers.AdamW(learning_rate=0.0001)


    model.compile(optimizer=optimizer,loss={'synergy':'mse','class':'categorical_crossentropy'})

    model.fit([train_d1,train_d2,train_c],[train_synergy,train_class],shuffle=True, epochs=epo, batch_size=batch_size,verbose=1 )
              

    return model



In [6]:
# train_d1=[train_f_drug1,train_graph1,train_finger_d1,train_target1]
# train_d2=[train_f_drug2,train_graph2,train_finger_d2,train_target2]
# train_c=[train_cell_line,train_copy,train_mutation,train_protematics]
# test_d1=[test_f_drug1,test_graph1,test_finger_d1,test_target1]
# test_d2=[test_f_drug2,test_graph2,test_finger_d2,test_target2]
# test_c=[test_cell_line,test_copy,test_mutation,test_protematics]



train_d1=train_target1
train_d2=train_target2
train_c=train_protematics
test_d1=test_target1
test_d2=test_target2
test_c=test_protematics

train_d1=tf.convert_to_tensor(train_d1)
test_d1=tf.convert_to_tensor(test_d1)
train_d2=tf.convert_to_tensor(train_d2)
test_d2=tf.convert_to_tensor(test_d2)
train_c=tf.convert_to_tensor(train_c)
test_c=tf.convert_to_tensor(test_c)

In [7]:

   
max_epoch =500
batch_size = 512 
earlyStop_patience = 20




model_att1= generate_network_att1(train_d1,train_d2,train_c)

train_class1 = keras.utils.to_categorical(train_class, num_classes=3)
test_class1 = keras.utils.to_categorical(test_class, num_classes=3)

model_att1=trainer_att1(model_att1, train_d1,train_d2,train_c,train_synergy,train_class1,max_epoch, batch_size,
                                earlyStop_patience,test_d1,test_d2,test_c,test_synergy,test_class1)

ap111,ap221= model_att1.predict( [test_d1,test_d2,test_c])

positive_negative_indices = np.where(test_class != 2)
ap221=ap221[positive_negative_indices]
test_class12=np.array(test_class)
test_class12=test_class12[positive_negative_indices]
ap221=ap221[:,1]


ap11=[]
test_synergy1=[]
l=len(ap111)
l1=int(l/2)
for i in range(l1) :
    ap=(ap111[i]+ap111[i+l1])/2
    ap11.append(ap)
    aap=(test_synergy[i]+test_synergy[i+l1])/2
    test_synergy1.append(aap)
    
    
ap22=[]
test_class1=[]
l=len(ap221)
l1=int(l/2)
for i in range(l1) :
    ap=(ap221[i]+ap221[i+l1])/2
    ap22.append(ap)
    aap=(test_class12[i]+test_class12[i+l1])/2
    test_class1.append(aap)
    
    
    
asynergy_error1=mean_squared_error(test_synergy1, ap11)
asynergy_error11=mean_absolute_error(test_synergy1, ap11)
asynergy_error21=r2_score(test_synergy1, ap11)

print("msynergy_mean_squared_error",asynergy_error1)
print("mclass_mean_squared_error",asynergy_error11)
print("msynergy_r2_score",asynergy_error21)

asynergy_pear1= pearsonr(test_synergy1, ap11)
asynergy_spear1= spearmanr(test_synergy1, ap11)
print("msynergy_pear",asynergy_pear1)
print("msynergy_spear",asynergy_spear1)

aap2=convert_tobin(ap22)

aclass_error1=roc_auc_score(test_class1, ap22)
aclass_error11=accuracy_score(test_class1, aap2)
aclass_error21=cohen_kappa_score(test_class1, aap2)

print("msclass_roc_curve",aclass_error1)
print("mclass_accuracy_scorer",aclass_error11)
print("mclass_cohen_kappa_score",aclass_error21)


aclass_pear1= precision_score(test_class1, aap2)
aclass_spear1= average_precision_score(test_class1, ap22)
print("mclass_precision_score",aclass_pear1)
print("mclass_average_precision_score",aclass_spear1)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 567)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 567)]                0         []                            
                                                                                                  
 train (InputLayer)          [(None, 256)]                0         []                            
                                                                                                  
 dense (Dense)               (None, 1024)                 581632    ['input_1[0][0]']             
                                                                                              

In [8]:
from IPython.display import FileLink 

np.savetxt('tarpro_cls4.csv', ap22 ,delimiter=',')
FileLink(r'tarpro_cls4.csv')    

/kaggle/working/tarpro_cls4.csv

In [9]:
np.savetxt('tarpro_syn4.csv', ap11 ,delimiter=',')
FileLink(r'tarpro_syn4.csv')

/kaggle/working/tarpro_syn4.csv

In [10]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


print(tf.__version__)
import keras as kr
print(kr.__version__)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import platform
print(platform.python_version())


print(np.__version__)
print(pd.__version__)

2.13.0
2.13.1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13521567510330604109
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16274030592
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16858313825578029128
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
xla_global_id: 416903419
]
3.10.12
1.24.3
1.3.5
